In [ ]:
!pip install flask

In [ ]:
#gives you a url that you can paste in browser when flask app is running to access a website
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#writing a simple flask app that will return the main.html page using render_template function (not JSON)
#main.html is stored in the Templates folder under Flask folder in google drive

from flask import Flask, jsonify, request
from flask import render_template
import os

#specifiying the folder paths for static and template folders & using those folders to make the flask app
static_folder_path = 'drive/My Drive/Flask/Static'
template_folder_path = 'drive/My Drive/Flask/Templates'
print(os.listdir("/content/drive/My Drive/Flask/Templates"))
app = Flask(__name__, static_folder=static_folder_path, template_folder=template_folder_path)

In [ ]:
#INSTALLING PYMILVUS

!pip install -U pymilvus
!pip install --upgrade pymilvus
!pip install "pymilvus[model]"

In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient("/content/drive/MyDrive/project_work.db")

In [ ]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

search_params = {
    "metric_type": "L2",    # match your index metric
    "params": {"nprobe": 10}
    }

In [ ]:
import os

In [ ]:
from posixpath import split
#the function to serve up the page
#in this instance, passing a message from Flask App to HTML
#the message "howdy" is passed in a message from Flask App to HTML
@app.route("/")
def main():
  return render_template("main.html",message="howdy")

@app.route("/page1")
def serve_page1():
    return render_template('page1.html')


@app.route("/page2")
def serve_page2():
    return render_template('page2.html')

@app.route("/image/<fileName>")
def serve_image(fileName):
    message = fileName
    img = '/Static/Images/Samples/' + fileName
    return render_template('Images.html', message=message, img=img)

@app.route("/video/<fileName>")
def serve_video(fileName):
    #message = "Video Route"
    message = fileName
    vid = '/Static/Videos/Samples/' + fileName
    return render_template('Video.html', message=message, vid=vid)


@app.route("/pdf/<fileName>")
def serve_pdf(fileName):
    #message = "Video Route"
    message = fileName
    pdf = '/Static/PDF/Samples/' + fileName
    return render_template('PDF.html', message=message, pdf=pdf)

@app.route("/text/<fileName>")
def serve_text(fileName):
    #message = "Video Route"
    message = fileName
    txt = '/Static/Text/Samples/' + fileName
    return render_template('Text.html', message=message, txt=txt)

@app.route("/index")
def serve_index():
    # hardcoding links. In your real project version, you will query
    # Milvus, iterate and create this array of dictionaries.

    #search milvus for the search string
    query_vector= embedding_fn.encode_queries(["How do I make brownies?"])
    res1 = milvus_client.search(
        collection_name="project_collection",
        data=query_vector,
        anns_field="embedding",
        search_params = search_params,
        limit=5, #returns top 5 similar results
        output_fields=["text","FilePath","FileExtension"],

    )
    links = []
    innerRes1 = res1[0]
    for dict in innerRes1:
      distance = dict.distance
      entityDict = dict.entity
      fileSummary = entityDict.text
      FilePath = entityDict.FilePath
      FileExtension = entityDict.FileExtension
      FileName = os.path.basename(FilePath)
      if FileExtension == ".jpg" or FileExtension == ".jpeg" or FileExtension == ".webp":
        links.append({'summary':fileSummary, 'endpoint':'serve_image', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".pdf":
        links.append({'summary':fileSummary, 'endpoint':'serve_pdf', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".mp4" or FileExtension ==".mov":
        links.append({'summary':fileSummary, 'endpoint':'serve_video', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".txt":
        links.append({'summary':fileSummary, 'endpoint':'serve_text', 'fileName': FileName, 'distance':distance})
    print(links)
    return render_template('index.html', links=links)

@app.route('/search', methods=['POST'])
def search():
  json = request.get_json()
  searchStr = json['searchText']
  fileType = json['fileType']
  print(fileType)
  query_vector= embedding_fn.encode_queries([searchStr])
  if fileType == "all":
    res1 = milvus_client.search(
        collection_name="project_collection",
        data=query_vector,
        anns_field="embedding",
        search_params = search_params,
        limit=5, #returns top 5 similar results
        output_fields=["text","FilePath","FileExtension"],

      )
  else:
    filter = 'FileExtension in ["'+fileType+'"]'
    res1 = milvus_client.search(
        collection_name="project_collection",
        data=query_vector,
        anns_field="embedding",
        filter=filter,
        search_params = search_params,
        limit=5, #returns top 5 similar results
        output_fields=["text","FilePath","FileExtension"],

      )
  links = []
  innerRes1 = res1[0]
  for dict in innerRes1:
      distance = dict.distance
      entityDict = dict.entity
      fileSummary = entityDict.text
      FilePath = entityDict.FilePath
      FileExtension = entityDict.FileExtension
      FileName = os.path.basename(FilePath)
      if FileExtension == ".jpg" or FileExtension == ".jpeg" or FileExtension == ".webp":
        links.append({'summary':fileSummary, 'endpoint':'serve_image', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".pdf":
        links.append({'summary':fileSummary, 'endpoint':'serve_pdf', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".mp4" or FileExtension ==".mov":
        links.append({'summary':fileSummary, 'endpoint':'serve_video', 'fileName': FileName, 'distance':distance})
      elif FileExtension == ".txt":
        links.append({'summary':fileSummary, 'endpoint':'serve_text', 'fileName': FileName, 'distance':distance})
  print(links)
  return render_template('index.html', links=links)
  # rest  is same as serve_index except that you now use searchStr
  # instead of a hard coded prompt.


if __name__ == "__main__":
    app.run()

In [ ]:
#links = [
        #{'summary': 'some text summary for image', 'endpoint': 'serve_image', 'fileName':'waterfall.jpg', 'distance':'1.234'},
        #{'summary': 'some text summary for video', 'endpoint': 'serve_video', 'fileName':'dance.mp4', 'distance':'1.434'},
        #{'summary': 'some text summary for PDF', 'endpoint': 'serve_pdf', 'fileName':'EasyRecipes.pdf', 'distance':'2.234'},
        #{'summary': 'some text summary for text', 'endpoint': 'serve_text', 'fileName':'peter_rabbit.txt', 'distance':'1.112'}
    #]